In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error,f1_score

import numpy as np
from random import seed
from random import randrange
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
%matplotlib inline

## Exploratory Data Analysis 

In [5]:
df = pd.read_csv('./Datasets/q3/train.csv')    
rows,cols=df.shape    
print(df.head())
print(df.describe())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         99.0    12099   Pave   NaN      IR1   
1   2          20       RL         89.0    13214   Pave   NaN      IR1   
2   3         120       RM         32.0     4500   Pave   NaN      Reg   
3   4         160       FV         30.0     3000   Pave  Pave      Reg   
4   5          20       RL          NaN     6897   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         HLS    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      6   2007     

Replace missing values 

In [7]:
print( df.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
for column in df.columns:    
    df[column].fillna(df[column].mode()[0], inplace=True)   

Observe the distribution of training data labels

In [18]:
for col in df.columns.values:
    print(col, df[col].unique())    

Id [   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15
   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30
   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45
   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60
   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75
   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90
   91   92   93   94   95   96   97   98   99  100  101  102  103  104  105
  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120
  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135
  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150
  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165
  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180
  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195
  196  19

In [22]:
print("No. of columns containing null values")
print(len(df.columns[df.isna().any()]))

print("No. of columns not containing null values")
print(len(df.columns[df.notna().all()]))

print("Total no. of columns in the dataframe")
print(len(df.columns))

No. of columns containing null values
19
No. of columns not containing null values
62
Total no. of columns in the dataframe
81


In [23]:
print("Columns containing null values")
print(df.columns[df.isna().any()].tolist())

Columns containing null values
['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [21]:
df.loc[:, df.isnull().any()]

LotFrontage Alley MasVnrType  MasVnrArea BsmtQual BsmtCond BsmtExposure  \
0           99.0   NaN    BrkFace       388.0       Ex       TA           Av   
1           89.0   NaN       None         0.0       Ex       TA           Gd   
2           32.0   NaN    BrkFace       443.0       Ex       Gd           No   
3           30.0  Pave      Stone        76.0       Gd       TA           Av   
4            NaN   NaN       None         0.0       TA       TA           No   
5           36.0   NaN      Stone       106.0       Gd       TA           No   
6           32.0   NaN    BrkFace       320.0       Ex       TA           No   
7            NaN   NaN       None         0.0       Gd       TA           No   
8           68.0   NaN       None         0.0       TA       TA           Av   
9           70.0   NaN       None         0.0       TA       TA           No   
10          60.0   NaN       None         0.0      NaN      NaN          NaN   
11          50.0   NaN       None         0.0       TA       TA           No   
12          58.0   NaN       None         0.0       Gd       TA           No   
13          70.0   NaN    BrkFace       192.0       Gd       TA           Av   
14          60.0   NaN       None         0.0       TA       TA           No   
15         120.0   NaN       None         0.0       TA       TA           Av   
16          51.0   NaN       None         0.0       TA       TA           No   
17          75.0   NaN       None         0.0       Gd       TA           Av   
18          60.0   NaN       None         0.0       TA       TA           No   
19          75.0   NaN      Stone        36.0       Gd       TA           Av   
20          90.0   NaN    BrkFace       256.0       Gd       TA           Gd   
21          71.0   NaN       None         0.0       TA       TA           No   
22         144.0   NaN       None         0.0       TA       TA           Gd   
23          64.0   NaN      Stone       275.0       Gd       TA           Av   
24         138.0   NaN       None         0.0       TA       TA           No   
25          73.0   NaN       None         0.0       Gd       TA           Av   
26          67.0   NaN    BrkFace       110.0       Gd       TA           No   
27         134.0   NaN    BrkFace       456.0       Gd       TA           Mn   
28          88.0   NaN      Stone       350.0       Ex       TA           Av   
29          60.0  Grvl       None         0.0       Fa       TA           No   
..           ...   ...        ...         ...      ...      ...          ...   
970         60.0   NaN       None         0.0       TA       TA           Mn   
971        107.0   NaN       None         0.0       Gd       TA           No   
972          NaN   NaN       None         0.0       Gd       TA           No   
973          NaN   NaN    BrkFace       170.0       Gd       TA           No   
974         57.0   NaN       None         0.0       TA       TA           No   
975         80.0   NaN       None         0.0      NaN      NaN          NaN   
976         21.0   NaN    BrkFace       408.0       TA       Fa           No   
977         92.0   NaN       None         0.0       Fa       TA           No   
978         60.0   NaN    BrkFace        16.0       Gd       TA           No   
979         67.0   NaN       None         0.0       TA       TA           No   
980         56.0   NaN    BrkFace       870.0       Ex       TA           No   
981         70.0   NaN       None         1.0       TA       TA           No   
982         96.0   NaN    BrkFace        97.0       TA       TA           Av   
983         85.0   NaN    BrkFace       108.0       Gd       TA           Gd   
984         70.0   NaN       None         0.0       Gd       TA           No   
985         62.0   NaN    BrkFace       472.0       Ex       TA           Gd   
986         21.0   NaN    BrkFace       510.0       TA       TA           No   
987          NaN   NaN    BrkFace       189.0       TA       TA           No   
988  

In [30]:
print('Value counts analysis')

for col in df.columns[df.isna().any()].tolist():
 
    percent_of_nulls=(sum(pd.isnull(df[col]))/rows)*100.0
    print('Percentage of nulls - ' + col ,percent_of_nulls)

Value counts analysis
Percentage of nulls - LotFrontage 17.5
Percentage of nulls - Alley 95.0
Percentage of nulls - MasVnrType 0.7000000000000001
Percentage of nulls - MasVnrArea 0.7000000000000001
Percentage of nulls - BsmtQual 3.0
Percentage of nulls - BsmtCond 3.0
Percentage of nulls - BsmtExposure 3.1
Percentage of nulls - BsmtFinType1 3.0
Percentage of nulls - BsmtFinType2 3.1
Percentage of nulls - Electrical 0.1
Percentage of nulls - FireplaceQu 46.400000000000006
Percentage of nulls - GarageType 5.3
Percentage of nulls - GarageYrBlt 5.3
Percentage of nulls - GarageFinish 5.3
Percentage of nulls - GarageQual 5.3
Percentage of nulls - GarageCond 5.3
Percentage of nulls - PoolQC 99.4
Percentage of nulls - Fence 80.80000000000001
Percentage of nulls - MiscFeature 96.39999999999999


Drop the columns that have more missing values

In [35]:
df.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'], axis = 1,inplace=True) 

Drop rows for GarageYrBlt as year is numeric and we may not want to use average or mode

In [36]:
df.dropna(subset=['GarageYrBlt'], how='all', inplace=True)

After removing columns and rows  that do not contribute

In [37]:
df.shape

(947, 76)

['LotFrontage',  'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType',  'GarageFinish', 'GarageQual', 
 'GarageCond']
 
 Replacing the missing values based on data type

In [44]:
df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace = True)

df['MasVnrType'].fillna(df['MasVnrType'].mode().values[0], inplace = True)

df['MasVnrArea'].fillna(df['MasVnrArea'].mean(), inplace = True)

df['BsmtQual'].fillna('UNK', inplace = True)

df['BsmtCond'].fillna('UNK', inplace = True)

df['BsmtExposure'].fillna('UNK', inplace = True)

df['BsmtFinType1'].fillna('UNK', inplace = True)

df['BsmtFinType2'].fillna('UNK', inplace = True)

df['Electrical'].fillna(df['Electrical'].mode().values[0], inplace = True)

df['GarageType'].fillna('UNK', inplace = True)

df['GarageFinish'].fillna('UNK', inplace = True)

df['GarageQual'].fillna('UNK', inplace = True)

df['GarageCond'].fillna('UNK', inplace = True)



Veryfying that missing values are addressed on full dataset

In [45]:
print("No. of columns containing null values")
print(len(df.columns[df.isna().any()]))

print("No. of columns not containing null values")
print(len(df.columns[df.notna().all()]))

print("Total no. of columns in the dataframe")
print(len(df.columns))

No. of columns containing null values
0
No. of columns not containing null values
76
Total no. of columns in the dataframe
76


In [46]:
print(df.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0   1          60       RL    99.000000    12099   Pave      IR1         Lvl   
1   2          20       RL    89.000000    13214   Pave      IR1         HLS   
2   3         120       RM    32.000000     4500   Pave      Reg         Lvl   
3   4         160       FV    30.000000     3000   Pave      Reg         Lvl   
4   5          20       RL    71.916452     6897   Pave      IR1         Lvl   

  Utilities LotConfig    ...     EnclosedPorch 3SsnPorch ScreenPorch PoolArea  \
0    AllPub    Inside    ...                 0         0           0        0   
1    AllPub    Inside    ...                 0         0           0        0   
2    AllPub       FR2    ...                 0         0           0        0   
3    AllPub    Inside    ...                 0         0           0        0   
4    AllPub    Corner    ...                 0         0           0        0   

  MiscVal MoSold  YrSold  SaleTy

In [52]:
df.head()

MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0          60       RL    99.000000    12099   Pave      IR1         Lvl   
1          20       RL    89.000000    13214   Pave      IR1         HLS   
2         120       RM    32.000000     4500   Pave      Reg         Lvl   
3         160       FV    30.000000     3000   Pave      Reg         Lvl   
4          20       RL    71.916452     6897   Pave      IR1         Lvl   

  Utilities LotConfig LandSlope    ...    EnclosedPorch 3SsnPorch ScreenPorch  \
0    AllPub    Inside       Gtl    ...                0         0           0   
1    AllPub    Inside       Gtl    ...                0         0           0   
2    AllPub       FR2       Gtl    ...                0         0           0   
3    AllPub    Inside       Gtl    ...                0         0           0   
4    AllPub    Corner       Gtl    ...                0         0           0   

  PoolArea MiscVal  MoSold  YrSold  SaleType  SaleCondition SalePrice  
0        0       0       6    2007        WD         Normal    354000  
1        0       0       5    2010        WD         Normal    378500  
2        0       0       3    2006        WD         Normal    153500  
3        0       0       6    2009       New        Partial    173733  
4        0       0       4    2010        WD         Normal    127000  

[5 rows x 75 columns]

In [53]:
X=df.drop('SalePrice',axis=1) #Predictors
y=df['SalePrice'] #Response

In [57]:
X.head()

MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0          60       RL    99.000000    12099   Pave      IR1         Lvl   
1          20       RL    89.000000    13214   Pave      IR1         HLS   
2         120       RM    32.000000     4500   Pave      Reg         Lvl   
3         160       FV    30.000000     3000   Pave      Reg         Lvl   
4          20       RL    71.916452     6897   Pave      IR1         Lvl   

  Utilities LotConfig LandSlope      ...      OpenPorchSF EnclosedPorch  \
0    AllPub    Inside       Gtl      ...              154             0   
1    AllPub    Inside       Gtl      ...               76             0   
2    AllPub       FR2       Gtl      ...              199             0   
3    AllPub    Inside       Gtl      ...              234             0   
4    AllPub    Corner       Gtl      ...              104             0   

  3SsnPorch ScreenPorch PoolArea  MiscVal  MoSold  YrSold  SaleType  \
0         0           0        0        0       6    2007        WD   
1         0           0        0        0       5    2010        WD   
2         0           0        0        0       3    2006        WD   
3         0           0        0        0       6    2009       New   
4         0           0        0        0       4    2010        WD   

  SaleCondition  
0        Normal  
1        Normal  
2        Normal  
3       Partial  
4        Normal  

[5 rows x 74 columns]

In [58]:
y[0]

354000

In [114]:
from regression_tree.supervised_learning import decision_tree
from regression_tree.supervised_learning.decision_tree import *

In [66]:
dtree_regressor =RegressionTree()

In [67]:
dtree_regressor.fit(X,y)

In [68]:
dtree_regressor.print_tree()

15:8? 
 T->42:1589? 
  T->3:35760? 
    T->2:118.0? 
        T->625000.0
        F->538000.0
    F->2:160.0? 
        T->745000.0
        F->755000.0
  F->15:9? 
    T->2:130.0? 
        T->2:313.0? 
                T->160000.0
                F->184750.0
        F->44:2822? 
                T->42:1174? 
                                T->0:75? 
                                                                T->475000.0
                                                                F->485000.0
                                F->2:107.0? 
                                                                T->582933.0
                                                                F->556581.0
                F->32:1280? 
                                T->44:2076? 
                                                                T->32:1767? 
                                                                                                                                T->555000.0
            

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Cleaning test data for missing values

In [88]:
df1 = pd.read_csv('./Datasets/q3/test.csv')  
rows1,_=df1.shape

df2 = pd.read_csv('./Datasets/q3/test_labels.csv',header=None)    
rows2,_=df2.shape


In [81]:
df1.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [89]:
df2.columns = ['FK','SalePrice']
#df1['SalePrice'] = df2.values 
df1['FK'] = df2['FK']
df1['SalePrice'] = df2['SalePrice'] 

In [90]:
df1.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1001          20       RL         80.0     9920   Pave   NaN      Reg   
1  1002          20       RL         60.0     7200   Pave   NaN      Reg   
2  1003          20       RL         93.0     9382   Pave   NaN      IR1   
3  1004          90       RL         64.0     6979   Pave   NaN      Reg   
4  1005          60       RL         65.0    15426   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolQC  Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub    ...        NaN  MnPrv         NaN       0      2   
1         Bnk    AllPub    ...        NaN  MnPrv         NaN       0      1   
2         Lvl    AllPub    ...        NaN    NaN         NaN       0      7   
3         Lvl    AllPub    ...        NaN  GdPrv        Shed     600      6   
4         Lvl    AllPub    ...        NaN  MnPrv         NaN       0      8   

  YrSold SaleType  SaleCondition    FK  SalePrice  
0   2010       WD         Normal  1001     128000  
1   2008       WD         Normal  1002     124900  
2   2008       WD         Normal  1003     191000  
3   2010       WD         Normal  1004     144000  
4   2009       WD         Normal  1005     231500  

[5 rows x 82 columns]

In [91]:
print("No. of columns containing null values")
print(len(df1.columns[df1.isna().any()]))

print("No. of columns not containing null values")
print(len(df1.columns[df1.notna().all()]))

print("Total no. of columns in the dataframe")
print(len(df1.columns))

No. of columns containing null values
18
No. of columns not containing null values
64
Total no. of columns in the dataframe
82


In [92]:
print('Value counts analysis')

for col in df1.columns[df1.isna().any()].tolist():
 
    percent_of_nulls=(sum(pd.isnull(df1[col]))/rows)*100.0
    print('Percentage of nulls - ' + col ,percent_of_nulls)
	

Value counts analysis
Percentage of nulls - LotFrontage 8.4
Percentage of nulls - Alley 41.9
Percentage of nulls - MasVnrType 0.1
Percentage of nulls - MasVnrArea 0.1
Percentage of nulls - BsmtQual 0.7000000000000001
Percentage of nulls - BsmtCond 0.7000000000000001
Percentage of nulls - BsmtExposure 0.7000000000000001
Percentage of nulls - BsmtFinType1 0.7000000000000001
Percentage of nulls - BsmtFinType2 0.7000000000000001
Percentage of nulls - FireplaceQu 22.6
Percentage of nulls - GarageType 2.8000000000000003
Percentage of nulls - GarageYrBlt 2.8000000000000003
Percentage of nulls - GarageFinish 2.8000000000000003
Percentage of nulls - GarageQual 2.8000000000000003
Percentage of nulls - GarageCond 2.8000000000000003
Percentage of nulls - PoolQC 45.9
Percentage of nulls - Fence 37.1
Percentage of nulls - MiscFeature 44.2


Drop the columns that have more missing values

In [93]:
df1.drop(['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'], axis = 1,inplace=True)

Drop rows for GarageYrBlt as year is numeric and we may not want to use average or mode

In [96]:
df1.dropna(subset=['GarageYrBlt'], how='all', inplace=True)


Replacing the missing values based on data type

In [97]:

df1['LotFrontage'].fillna(df1['LotFrontage'].mean(), inplace = True)

df1['MasVnrType'].fillna(df1['MasVnrType'].mode().values[0], inplace = True)

df1['MasVnrArea'].fillna(df1['MasVnrArea'].mean(), inplace = True)

df1['BsmtQual'].fillna('UNK', inplace = True)

df1['BsmtCond'].fillna('UNK', inplace = True)

df1['BsmtExposure'].fillna('UNK', inplace = True)

df1['BsmtFinType1'].fillna('UNK', inplace = True)

df1['BsmtFinType2'].fillna('UNK', inplace = True)

df1['Electrical'].fillna(df1['Electrical'].mode().values[0], inplace = True)

df1['GarageType'].fillna('UNK', inplace = True)

df1['GarageFinish'].fillna('UNK', inplace = True)

df1['GarageQual'].fillna('UNK', inplace = True)

df1['GarageCond'].fillna('UNK', inplace = True)


Veryfying that missing values are addressed on full dataset

In [98]:
print("No. of columns containing null values")
print(len(df1.columns[df1.isna().any()]))

print("No. of columns not containing null values")
print(len(df1.columns[df1.notna().all()]))

print("Total no. of columns in the dataframe")
print(len(df1.columns))

No. of columns containing null values
0
No. of columns not containing null values
77
Total no. of columns in the dataframe
77


In [106]:
X_test=df1.drop('SalePrice',axis=1) #Predictors
y_test=df1['SalePrice'] #Response

In [107]:
X_test=X_test.values

In [109]:
X_test

array([[1001, 20, 'RL', ..., 'WD', 'Normal', 1001],
       [1002, 20, 'RL', ..., 'WD', 'Normal', 1002],
       [1003, 20, 'RL', ..., 'WD', 'Normal', 1003],
       ..., 
       [1458, 20, 'RL', ..., 'WD', 'Normal', 1458],
       [1459, 20, 'RL', ..., 'WD', 'Normal', 1459],
       [1460, 20, 'RL', ..., 'WD', 'Normal', 1460]], dtype=object)